# ADM-HW4: GROUP #10

#### Xavier Del Giudice


| STUDENT |   ID    |                 E-mail                  |
| :-: |:-------:|:---------------------------------------:|
| Xavier Del Giudice | 1967219 | delgiudice.1967219@studenti.uniroma1.it |
|  |  |                                         |
|  |  |                       |

## Import and Install required packages

In [ ]:
%pip install pandas numpy

In [ ]:
import pandas as pd
import numpy as np

# 1. Recommendation System with LSH

## 1.1 Data Preparation

## 1.2 Minhash Signatures

## 1.3 Locality-Sensitive Hashing (LSH)

### 1. Bucket Creation

### 2. Query

### 3. Movie Recommendation Logic

### 4. Final Recommendation

# 2. Grouping Movies Together!

## 2.1 Feature Engineering

## 2.2 Choose your features (variables)!

## 2.3 Clustering

## 2.4 Best Algorithm

# 3. Bonus Question

# 4. Algorithmic Question